In [1]:
import torch
import timm
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from torchvision.datasets import ImageFolder
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import random
import warnings
##from tqdm import tqdm 
warnings.filterwarnings("ignore")

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    ##torch.backends.cudnn.deterministic = False
    ##torch.backends.cudnn.benchmark = True
    torch.use_deterministic_algorithms(True, warn_only=True)
    print(f"[INFO] All seeds set to {seed}")

seed_everything(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

# Configuration
##DATA_DIR = "/kaggle/input/mapsimgtxt/spammultimodal"  # Update with dataset path
DATA_DIR = "/kaggle/input/ekafnewsforkhawla/sorted_imagesOur" 

BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define models with specific input sizes      
MODELS = {"vit_base_patch32_224": (224, 224),    
          "deit_tiny_patch16_224.fb_in1k": (224, 224),    
          "beit_base_patch16_224.in22k_ft_in22k": (224, 224),    
          "cait_xxs24_224.fb_dist_in1k": (224, 224),
          "timm/crossvit_18_dagger_408.in1k": (408, 408)
}

# Load dataset
dataset = ImageFolder(DATA_DIR)
num_classes = len(dataset.classes)
labels = [sample[1] for sample in dataset.samples]  # Extract labels properly
kf = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

def free_gpu_memory():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

def train_model(model_name, input_size):
    print(f"Training {model_name}...")
    
    # Free GPU memory before starting a new model
    
    
    # Define data transformations
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor()
    ])
    dataset.transform = transform  # Apply transformation to dataset
    
        
    all_preds1, all_labels1 = [], []  # Accumulate results across folds
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset.samples, dataset.targets)):
        free_gpu_memory()
        model = timm.create_model(model_name, pretrained=True, num_classes=num_classes).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(model.parameters(), lr=LR)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)        
        print(f"Fold {fold+1}/5")
        train_subsampler = Subset(dataset, train_idx)
        val_subsampler = Subset(dataset, val_idx)
        
        train_loader = DataLoader(train_subsampler, batch_size=BATCH_SIZE,generator=g, shuffle=True)
        val_loader = DataLoader(val_subsampler, batch_size=BATCH_SIZE, generator=g,shuffle=False)
        
        for epoch in range(EPOCHS):
            model.train()
            running_loss, correct, total = 0.0, 0, 0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)
            
            for images, labels in progress_bar:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                progress_bar.set_postfix(loss=running_loss / len(train_loader), acc=100 * correct / total)
            
            print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
            scheduler.step(running_loss / len(train_loader))
        
        # Validation
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validating", leave=False):
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_preds1.extend(preds.cpu().numpy())
                all_labels1.extend(labels.cpu().numpy())
            
        print(f"Fold {fold+1} Accuracy: {accuracy_score(all_labels, all_preds):.4f}")
        print(classification_report(all_labels, all_preds, digits=4, zero_division=0))  
        print(confusion_matrix(all_labels, all_preds))
    
    print(f"\n✅✅✅✅✅✅✅✅✅✅✅ {model_name}✅✅✅✅✅✅✅✅✅✅✅✅✅")
    print(classification_report(all_labels1, all_preds1, digits=4, zero_division=0))  
    print(confusion_matrix(all_labels1, all_preds1))
    print(f"\n✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅")

# Train each model with corresponding input size
for model_name, input_size in MODELS.items():
    train_model(model_name, input_size)


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

[INFO] All seeds set to 42
Training vit_base_patch32_224...


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Fold 1/5


Epoch 1, Loss: 0.5391, Accuracy: 74.40%


...

Fold 4 Accuracy: 0.7098
              precision    recall  f1-score   support

           0     0.3984    0.1875    0.2550       272
           1     0.7542    0.8980    0.8198       755

    accuracy                         0.7098      1027
   macro avg     0.5763    0.5428    0.5374      1027
weighted avg     0.6600    0.7098    0.6702      1027

[[ 51 221]
 [ 77 678]]
Fold 5/5


Epoch 1, Loss: 0.5401, Accuracy: 74.87%


Epoch 2, Loss: 0.3922, Accuracy: 83.22%


Epoch 3, Loss: 0.2071, Accuracy: 92.58%


Fold 5 Accuracy: 0.6650
              precision    recall  f1-score   support

           0     0.3000    0.1985    0.2389       272
           1     0.7426    0.8331    0.7853       755

    accuracy                         0.6650      1027
   macro avg     0.5213    0.5158    0.5121      1027
weighted avg     0.6254    0.6650    0.6406      1027

[[ 54 218]
 [126 629]]

✅✅✅✅✅✅✅✅✅✅✅ timm/crossvit_18_dagger_408.in1k✅✅✅✅✅✅✅✅✅✅✅✅✅
              precision    recall  f1-score   support

           0     0.3555    0.2513    0.2944      1361
           1     0.7560    0.8358    0.7939      3777

    accuracy                         0.6810      5138
   macro avg     0.5557    0.5436    0.5442      5138
weighted avg     0.6499    0.6810    0.6616      5138

[[ 342 1019]
 [ 620 3157]]

✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅✅
